In [8]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.path.dirname("."), '..'))
sys.path.insert(0, parent_dir)
from bsmodel import BSModel
import numpy as np
import matplotlib.pyplot as plt
import bridgestan as bs
from scipy.stats import norm
from ipywidgets import interact, FloatSlider

In [9]:
homedir = "/Users/edward/" # "/mnt/c/Users/jccer/"
bs.set_bridgestan_path(homedir + "bridgestan")

In [10]:
bs_model1 = BSModel(stan_file = homedir + "mcmcexperiments/stan/one_exponential.stan",
                   data_file = homedir + "mcmcexperiments/stan/one_exponential.json")
bs_model2 = BSModel(stan_file = homedir + "mcmcexperiments/stan/one_normal.stan",
                   data_file = homedir + "mcmcexperiments/stan/one_normal.json")
bs_model3 = BSModel(stan_file = homedir + "mcmcexperiments/stan/one_t.stan",
                   data_file = homedir + "mcmcexperiments/stan/one_t.json")

/Users/edward/venvs/py3/lib/python3.12/site-packages/bridgestan/model.py:177: RuntimeWarning: The version of the compiled model does not match the version of the Python package. Consider recompiling the model.
  warnings.warn(
/Users/edward/venvs/py3/lib/python3.12/site-packages/bridgestan/model.py:177: RuntimeWarning: The version of the compiled model does not match the version of the Python package. Consider recompiling the model.
  warnings.warn(


In [11]:
class SinhArcsinhDistribution():
    def __init__(self, m = 0.0, s = 0.0, d = 0.0, e = 0.0):
        self.m = m
        self.s = s
        self.d = d
        self.e = e

    def logpdf(self, x):
        m, s, d, e = self.m, self.s, self.d, self.e
        z = (x - m) / s
        out = -0.5 * np.log(2 * np.pi)
        out += -0.25 * (np.cosh(2 * d * np.arcsinh(z) - 2 * e) - 1)
        out += np.log(np.cosh(d * np.arcsinh(z) - e))
        out += np.log(d) - np.log(s)
        out += -0.5 * np.log(1 + z**2)
        return out

    def pdf(self, x):
        return np.exp(self.logpdf(x))

In [12]:
mu_slider = FloatSlider(value = 0.0, min = -10.0, max = 10.0, step = 0.01, description = 'μ')
sigma_slider = FloatSlider(value = 0.01, min = 0.01, max = 10.0, step = 0.01, description = 'σ')
m_slider = FloatSlider(value = 0, min = -10, max = 10, step = 0.01, description = 'm')
s_slider = FloatSlider(value = 1, min = 0.01, max = 10, step = 0.01, description = 's')
d_slider = FloatSlider(value = 1, min = 0.01, max = 10, step = 0.01, description = 'd')
e_slider = FloatSlider(value = 0, min = -10, max = 10, step = 0.01, description = 'e')

In [20]:
xx = np.linspace(-10, 10, 301)

def plot_combined(mu = 0.0, sigma = 1.0, m = 0.0, s = 0.0, d = 0.0, e = 0.0):
    x = np.linspace(mu - 5 * sigma, mu + 5 * sigma, 1000)
    NormalDistribution = norm(loc = mu, scale = sigma)
    fy = NormalDistribution.pdf(x)
    SinhArcsinh = SinhArcsinhDistribution(m = m, s = s, d = d, e = e)
    qy = SinhArcsinh.pdf(x)

    plt.figure(figsize = (8, 5))
    plt.plot(x, fy, label = f'Normal: μ = {mu: .2f}, σ = {sigma: .2f}', color = 'blue')
    plt.plot(x, qy, label = 'SAS', color = 'green')
    plt.plot(xx, fxx, label = "BridgeStan model")
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title('Bridgestan Model with Normal Distribution and SAS Distribution')
    plt.grid(True)
    plt.minorticks_on()
    plt.legend()
    plt.show()

In [21]:
fxx = [np.exp(bs_model1.log_density(np.array([xxn]), propto = False)) for xxn in xx]

interact(plot_combined,
         m = m_slider,
         s = s_slider,
         d = d_slider,
         e = e_slider,
         mu = mu_slider,
         sigma = sigma_slider)

interactive(children=(FloatSlider(value=0.0, description='μ', max=10.0, min=-10.0, step=0.01), FloatSlider(val…

<function __main__.plot_combined(mu=0.0, sigma=1.0, m=0.0, s=0.0, d=0.0, e=0.0)>

In [22]:

fxx = [np.exp(bs_model2.log_density(np.array([xxn]), propto = False)) for xxn in xx]

interact(plot_combined,
         m = m_slider,
         s = s_slider,
         d = d_slider,
         e = e_slider,
         mu = mu_slider,
         sigma = sigma_slider)

interactive(children=(FloatSlider(value=-3.389999999999999, description='μ', max=10.0, min=-10.0, step=0.01), …

<function __main__.plot_combined(mu=0.0, sigma=1.0, m=0.0, s=0.0, d=0.0, e=0.0)>

In [23]:
fxx = [np.exp(bs_model3.log_density(np.array([xxn]), propto = False)) for xxn in xx]

interact(plot_combined,
         m = m_slider,
         s = s_slider,
         d = d_slider,
         e = e_slider,
         mu = mu_slider,
         sigma = sigma_slider)

interactive(children=(FloatSlider(value=-3.389999999999999, description='μ', max=10.0, min=-10.0, step=0.01), …

<function __main__.plot_combined(mu=0.0, sigma=1.0, m=0.0, s=0.0, d=0.0, e=0.0)>